In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
#     eQs_logits = tf.reshape(eQs, shape=[-1])
#     aQs_logits = tf.reshape(aQs, shape=[-1])
#     #################################################### s'', Q' = ~env(s', ~a')
#     next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
#                                                                     labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
# #     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
# #     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
# #                               reuse=True)
# #     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
# #                                    action_size=action_size, state_size=state_size, reuse=True)
# #     aQs_logits = aQs2_logits # pervious one
# #     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# # #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# # #                                                                     labels=tf.zeros_like(rates))) # min
# # #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# # #                                                                      labels=tf.ones_like(rates))) # max
# #     ###################################################### Q(s,a)= r + Q'(s',a')
# #     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
# #     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
# #     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
# #     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
# #     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
# #                               reuse=True)
# #     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
# #                                    action_size=action_size, state_size=state_size, reuse=True)
# #     aQs_logits = aQs2_logits # pervious one
# #     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# # #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# # #                                                                     labels=tf.zeros_like(rates))) # min
# # #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# # #                                                                      labels=tf.ones_like(rates))) # max
# #     ###################################################### Q(s,a)= r + Q'(s',a')
# #     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
# #     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
# #     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:0.7390 eloss:0.0035 aloss2:0.6954 exploreP:0.9980
Episode:1 meanR:20.0000 R:20.0000 rate:0.0400 aloss:0.7459 eloss:0.0046 aloss2:0.7002 exploreP:0.9960
Episode:2 meanR:23.6667 R:31.0000 rate:0.0620 aloss:0.7404 eloss:0.0029 aloss2:0.6957 exploreP:0.9930
Episode:3 meanR:22.5000 R:19.0000 rate:0.0380 aloss:0.7331 eloss:0.0028 aloss2:0.6960 exploreP:0.9911
Episode:4 meanR:21.2000 R:16.0000 rate:0.0320 aloss:0.7431 eloss:0.0001 aloss2:0.6990 exploreP:0.9896
Episode:5 meanR:20.6667 R:18.0000 rate:0.0360 aloss:0.7329 eloss:0.0033 aloss2:0.6975 exploreP:0.9878
Episode:6 meanR:20.2857 R:18.0000 rate:0.0360 aloss:0.7277 eloss:0.0022 aloss2:0.6959 exploreP:0.9860
Episode:7 meanR:20.1250 R:19.0000 rate:0.0380 aloss:0.7157 eloss:0.0018 aloss2:0.6952 exploreP:0.9842
Episode:8 meanR:21.3333 R:31.0000 rate:0.0620 aloss:0.7192 eloss:0.0002 aloss2:0.6960 exploreP:0.9812
Episode:9 meanR:22.0000 R:28.0000 rate:0.0560 aloss:0.7153 eloss:0.0026 aloss2:0.6

Episode:80 meanR:27.3580 R:26.0000 rate:0.0520 aloss:0.6873 eloss:-0.0020 aloss2:0.7159 exploreP:0.8032
Episode:81 meanR:27.5000 R:39.0000 rate:0.0780 aloss:0.6852 eloss:-0.0023 aloss2:0.7171 exploreP:0.8001
Episode:82 meanR:27.7470 R:48.0000 rate:0.0960 aloss:0.7020 eloss:-0.0023 aloss2:0.7180 exploreP:0.7964
Episode:83 meanR:27.5952 R:15.0000 rate:0.0300 aloss:0.6755 eloss:-0.0020 aloss2:0.7184 exploreP:0.7952
Episode:84 meanR:27.7529 R:41.0000 rate:0.0820 aloss:0.6801 eloss:-0.0023 aloss2:0.7182 exploreP:0.7920
Episode:85 meanR:27.9884 R:48.0000 rate:0.0960 aloss:0.6765 eloss:-0.0024 aloss2:0.7199 exploreP:0.7882
Episode:86 meanR:28.0115 R:30.0000 rate:0.0600 aloss:0.6806 eloss:-0.0028 aloss2:0.7197 exploreP:0.7859
Episode:87 meanR:28.0795 R:34.0000 rate:0.0680 aloss:0.6873 eloss:-0.0026 aloss2:0.7203 exploreP:0.7833
Episode:88 meanR:28.5843 R:73.0000 rate:0.1460 aloss:0.6827 eloss:-0.0028 aloss2:0.7224 exploreP:0.7776
Episode:89 meanR:29.3444 R:97.0000 rate:0.1940 aloss:0.6842 elos

Episode:159 meanR:48.8800 R:25.0000 rate:0.0500 aloss:0.7733 eloss:-0.0084 aloss2:0.8709 exploreP:0.5400
Episode:160 meanR:48.9000 R:25.0000 rate:0.0500 aloss:0.7653 eloss:-0.0055 aloss2:0.8698 exploreP:0.5386
Episode:161 meanR:49.5000 R:93.0000 rate:0.1860 aloss:0.8038 eloss:-0.0023 aloss2:0.8646 exploreP:0.5337
Episode:162 meanR:49.4600 R:59.0000 rate:0.1180 aloss:0.8412 eloss:0.0002 aloss2:0.8562 exploreP:0.5307
Episode:163 meanR:50.1700 R:134.0000 rate:0.2680 aloss:0.7935 eloss:-0.0062 aloss2:0.8753 exploreP:0.5237
Episode:164 meanR:49.8900 R:38.0000 rate:0.0760 aloss:0.8280 eloss:-0.0097 aloss2:0.8795 exploreP:0.5218
Episode:165 meanR:50.1400 R:49.0000 rate:0.0980 aloss:0.8150 eloss:-0.0029 aloss2:0.8770 exploreP:0.5193
Episode:166 meanR:50.1100 R:79.0000 rate:0.1580 aloss:0.8334 eloss:-0.0075 aloss2:0.8786 exploreP:0.5153
Episode:167 meanR:51.1300 R:119.0000 rate:0.2380 aloss:0.7863 eloss:-0.0078 aloss2:0.8872 exploreP:0.5093
Episode:168 meanR:51.2400 R:40.0000 rate:0.0800 aloss:

Episode:238 meanR:41.9800 R:10.0000 rate:0.0200 aloss:0.7583 eloss:-0.0085 aloss2:0.9228 exploreP:0.4088
Episode:239 meanR:41.0500 R:9.0000 rate:0.0180 aloss:0.7735 eloss:-0.0063 aloss2:0.9161 exploreP:0.4084
Episode:240 meanR:40.8700 R:11.0000 rate:0.0220 aloss:0.7327 eloss:-0.0090 aloss2:0.9198 exploreP:0.4080
Episode:241 meanR:39.9000 R:15.0000 rate:0.0300 aloss:0.8465 eloss:-0.0100 aloss2:0.9285 exploreP:0.4074
Episode:242 meanR:39.9700 R:61.0000 rate:0.1220 aloss:0.8387 eloss:-0.0084 aloss2:0.9210 exploreP:0.4050
Episode:243 meanR:39.7600 R:13.0000 rate:0.0260 aloss:0.8298 eloss:-0.0098 aloss2:0.9134 exploreP:0.4045
Episode:244 meanR:39.2400 R:10.0000 rate:0.0200 aloss:0.8707 eloss:-0.0179 aloss2:0.9461 exploreP:0.4041
Episode:245 meanR:38.3800 R:11.0000 rate:0.0220 aloss:0.7764 eloss:-0.0136 aloss2:0.9255 exploreP:0.4036
Episode:246 meanR:37.9600 R:10.0000 rate:0.0200 aloss:0.7897 eloss:-0.0143 aloss2:0.9238 exploreP:0.4032
Episode:247 meanR:37.2700 R:9.0000 rate:0.0180 aloss:0.8

Episode:317 meanR:62.5700 R:10.0000 rate:0.0200 aloss:0.7017 eloss:-0.0091 aloss2:1.2534 exploreP:0.2290
Episode:318 meanR:62.5500 R:8.0000 rate:0.0160 aloss:0.7198 eloss:-0.0117 aloss2:1.2209 exploreP:0.2288
Episode:319 meanR:62.5600 R:11.0000 rate:0.0220 aloss:0.8174 eloss:-0.0022 aloss2:1.2113 exploreP:0.2286
Episode:320 meanR:62.5600 R:9.0000 rate:0.0180 aloss:0.7172 eloss:-0.0110 aloss2:1.2625 exploreP:0.2284
Episode:321 meanR:62.5700 R:11.0000 rate:0.0220 aloss:0.7450 eloss:-0.0070 aloss2:1.2437 exploreP:0.2282
Episode:322 meanR:62.2900 R:11.0000 rate:0.0220 aloss:0.7333 eloss:-0.0064 aloss2:1.1766 exploreP:0.2279
Episode:323 meanR:62.3000 R:9.0000 rate:0.0180 aloss:0.7146 eloss:-0.0112 aloss2:1.2483 exploreP:0.2277
Episode:324 meanR:62.2600 R:9.0000 rate:0.0180 aloss:0.7259 eloss:-0.0080 aloss2:1.2351 exploreP:0.2275
Episode:325 meanR:62.2400 R:9.0000 rate:0.0180 aloss:0.7229 eloss:-0.0134 aloss2:1.2458 exploreP:0.2273
Episode:326 meanR:62.1700 R:8.0000 rate:0.0160 aloss:0.7547 

Episode:395 meanR:82.1200 R:51.0000 rate:0.1020 aloss:0.7179 eloss:-0.0345 aloss2:1.5601 exploreP:0.1494
Episode:396 meanR:82.6900 R:81.0000 rate:0.1620 aloss:0.7149 eloss:-0.0260 aloss2:1.5309 exploreP:0.1483
Episode:397 meanR:83.5100 R:186.0000 rate:0.3720 aloss:0.7087 eloss:-0.0173 aloss2:1.5149 exploreP:0.1457
Episode:398 meanR:81.8300 R:94.0000 rate:0.1880 aloss:0.7210 eloss:-0.0049 aloss2:1.5134 exploreP:0.1445
Episode:399 meanR:81.4100 R:96.0000 rate:0.1920 aloss:0.7345 eloss:-0.0041 aloss2:1.5587 exploreP:0.1432
Episode:400 meanR:80.3900 R:22.0000 rate:0.0440 aloss:0.7270 eloss:-0.0120 aloss2:1.4641 exploreP:0.1429
Episode:401 meanR:80.4600 R:19.0000 rate:0.0380 aloss:0.8083 eloss:-0.0128 aloss2:1.5862 exploreP:0.1426
Episode:402 meanR:78.4900 R:21.0000 rate:0.0420 aloss:0.7583 eloss:-0.0173 aloss2:1.5611 exploreP:0.1423
Episode:403 meanR:74.5600 R:107.0000 rate:0.2140 aloss:0.7485 eloss:-0.0136 aloss2:1.4897 exploreP:0.1409
Episode:404 meanR:73.7600 R:105.0000 rate:0.2100 alos

Episode:474 meanR:49.0000 R:72.0000 rate:0.1440 aloss:0.7295 eloss:-0.0053 aloss2:1.4926 exploreP:0.1101
Episode:475 meanR:48.8700 R:43.0000 rate:0.0860 aloss:0.7373 eloss:-0.0054 aloss2:1.5186 exploreP:0.1096
Episode:476 meanR:49.2200 R:79.0000 rate:0.1580 aloss:0.7619 eloss:0.0086 aloss2:1.4647 exploreP:0.1089
Episode:477 meanR:47.5900 R:10.0000 rate:0.0200 aloss:0.7324 eloss:-0.0123 aloss2:1.5124 exploreP:0.1088
Episode:478 meanR:47.0600 R:80.0000 rate:0.1600 aloss:0.7818 eloss:0.0031 aloss2:1.5164 exploreP:0.1080
Episode:479 meanR:46.4800 R:65.0000 rate:0.1300 aloss:0.7732 eloss:-0.0079 aloss2:1.5149 exploreP:0.1073
Episode:480 meanR:45.9800 R:59.0000 rate:0.1180 aloss:0.7425 eloss:-0.0218 aloss2:1.4711 exploreP:0.1068
Episode:481 meanR:46.0100 R:108.0000 rate:0.2160 aloss:0.7423 eloss:-0.0193 aloss2:1.4992 exploreP:0.1057
Episode:482 meanR:46.2500 R:159.0000 rate:0.3180 aloss:0.7182 eloss:-0.0259 aloss2:1.4970 exploreP:0.1042
Episode:483 meanR:45.8700 R:115.0000 rate:0.2300 aloss:

Episode:552 meanR:75.9000 R:30.0000 rate:0.0600 aloss:0.7626 eloss:-0.0119 aloss2:1.4910 exploreP:0.0601
Episode:553 meanR:76.2200 R:41.0000 rate:0.0820 aloss:0.7648 eloss:-0.0107 aloss2:1.4503 exploreP:0.0598
Episode:554 meanR:77.8000 R:172.0000 rate:0.3440 aloss:0.7449 eloss:-0.0191 aloss2:1.4472 exploreP:0.0590
Episode:555 meanR:79.4000 R:172.0000 rate:0.3440 aloss:0.7106 eloss:-0.0193 aloss2:1.4873 exploreP:0.0582
Episode:556 meanR:79.5000 R:21.0000 rate:0.0420 aloss:0.6758 eloss:-0.0158 aloss2:1.4244 exploreP:0.0581
Episode:557 meanR:80.4500 R:104.0000 rate:0.2080 aloss:0.7346 eloss:-0.0130 aloss2:1.4766 exploreP:0.0576
Episode:558 meanR:83.1300 R:277.0000 rate:0.5540 aloss:0.7364 eloss:-0.0149 aloss2:1.4556 exploreP:0.0563
Episode:559 meanR:84.1800 R:115.0000 rate:0.2300 aloss:0.7430 eloss:-0.0212 aloss2:1.4959 exploreP:0.0557
Episode:560 meanR:84.5000 R:41.0000 rate:0.0820 aloss:0.7341 eloss:-0.0193 aloss2:1.4893 exploreP:0.0555
Episode:561 meanR:85.0100 R:61.0000 rate:0.1220 al

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
